In [136]:
import numpy as np
import scipy as sp
import scipy.linalg as linalg

In [123]:
def backward_sub(m, b):
    n = m.shape[0]
    assert m.shape == (n, n)
    assert b.shape == (n, )
    
    m2 = m.copy()
    x = b.copy()
    for i in range(n-1, -1, -1):
        #print(f"step {i}")
        #print(f"m2: {m2}")
        #print(f"b: {x}")
        x[i] = x[i] / m2[i, i]
        #print(x)
        x[:i] -= m2[:i, i] * x[i]
        #print(x)
    #x[0] = x[0] / m2[0, 0]
    
    return x

def forward_sub(m, b):
    n = m.shape[0]
    assert m.shape == (n, n)
    assert b.shape == (n, )
    
    x = b.copy()
    for i in range(n):      
        x[i:] -= m[i:, i] * b[i] / x[i]
    
    return x

In [89]:
def LU(m):
    n = m.shape[0]
    assert m.shape == (n, n)
    
    P = np.identity(n)
    
    L = np.identity(n)
    U = m.copy()
    
    if False:
        diag = [U[i, i] for i in range(n)]
        while 0 in diag:
            #find couple of rows to swap
            pass
            diag = [U[i, i] for i in range(n)]
    
    for i in range(n):
        #print(f"i: {i}")
        #print(f"ut {U}")
        L[i, i] = 1
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            #print(f"U[j, i:] -= L[j, i] * U[i, i:]")
            #print(f"{U[j, i:]} -= {L[j, i] * U[i, i:]}")
            U[j, i:] -= L[j, i] * U[i, i:]
    
    return L, U, P

In [99]:
def gauss_redux(m, b):
    n = m.shape[0]
    assert m.shape == (n, n)
    assert b.shape == (n,)
    
    U = np.empty((n, n+1))
    U[:, :n] = m
    U[:, n] = b
    
    if False:
        diag = [U[i, i] for i in range(n)]
        while 0 in diag:
            #find couple of rows to swap
            pass
            diag = [U[i, i] for i in range(n)]
    
    for i in range(n):
        #print(f"i: {i}")
        #print(f"ut {U}")
        #L[i, i] = 1
        for j in range(i+1, n):
            #L[j, i] = U[j, i] / U[i, i]
            #print(f"U[j, i:] -= L[j, i] * U[i, i:]")
            #print(f"{U[j, i:]} -= {L[j, i] * U[i, i:]}")
            U[j, i] = 0
            U[j, i:] -= (U[j, i] / U[i, i]) * U[i, i:]
    
    return U

In [143]:
n = 20
a = 100 + 100 * np.random.random((n, n))
b = 100 + 100 * np.random.random(n)

l, u, p = LU(a)
#print(f"(l@u) - a: {(l@u) - a}")
pp, ll, uu = linalg.lu(a)

print(f"max error mine: {np.max(np.abs((l@u) - a))}")
print(f"max error scipy: {np.max(np.abs((pp@ll@uu) - a))}")

print(f"(l@u) - a: {(l@u) - a}")
print(f"(pp@ll@uu) - a: {(pp@ll@uu) - a}")

print(pp @ll - l)

#print(f"a: {a}")
#print(f"b: {b}")

x = gauss_redux(a, b)
#print(x)
y = x[n-1, n]/x[n-1, n-1]
print(f"expected x[{n-1}]: {y}")
y2 = (x[n-2, n] - x[n-2, n-1] * y)/x[n-2, n-2]
print(f"expected x[{n-2}]: {y2}")
x = backward_sub(x[:, :n], x[:, n])
print(f"x: {x}")
(a @ x) - b
print(f"(a @ x) - b: {(a @ x) - b}")

max error mine: 7.673861546209082e-13
max error scipy: 7.105427357601002e-14
(l@u) - a: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.42108547e-14  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0

In [144]:
x, p = gauss_reduction(np.identity(n), b)
x, p

m: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
b: [0.82707434 0.82035953 0.21066007 0.51997706]


(array([0.82707434, 0.82035953, 0.21066007, 0.51997706]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]))